In [2]:
1e-4

0.0001

In [1]:
[] is None

False

In [8]:
[[]] == [
    [ ]
    ]

True

In [7]:
[] == [ ]

True

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['HUGGINGFACE_CACHE'] = '/share/huggingface/'
import sys
import json
sys.path.append('../')
import argparse
from easyeditor import LoRAHyperParams
from easyeditor import BaseEditor
from easyeditor import KnowEditDataset

data_dir = '../dataset/ccks2024_know_edit/ZsRE-test-all.json'
train_data_path = None
ds_size, data_type, = 326, 'zsre'
hparams_dir = '../hparams/LoRA/Meta-Llama-3-8B-Instruct'
metrics_save_dir = './EasyEditCache/metrics'

In [ ]:
datas = KnowEditDataset(data_dir,size=ds_size)
if data_type == 'counterfact' or data_type == 'recent' or data_type == 'zsre':
    prompts=[data['prompt'] for data in datas]
    subjects=[data['subject'] for data in datas]
    target_new = [data['target_new'] for data in datas]
    
    portability_r =[data['portability_r'] for data in datas]
    portability_s =[data['portability_s'] for data in datas]
    portability_l =[data['portability_l'] for data in datas]

    portability_reasoning_prompts=[]
    portability_reasoning_ans=[]
    portability_Logical_Generalization_prompts=[]
    portability_Logical_Generalization_ans=[]
    portability_Subject_Aliasing_prompts=[]
    portability_Subject_Aliasing_ans=[]
    
    portability_data = [portability_r,portability_s,portability_l]
    portability_prompts = [portability_reasoning_prompts,portability_Subject_Aliasing_prompts,portability_Logical_Generalization_prompts]
    portability_answers = [portability_reasoning_ans,portability_Subject_Aliasing_ans,portability_Logical_Generalization_ans]
    for data, portable_prompts, portable_answers in zip(portability_data,portability_prompts,portability_answers):
        for item in data:
            if item is None:
                portable_prompts.append(None)
                portable_answers.append(None)
            else:
                temp_prompts = []
                temp_answers = []
                for pr in item:
                    prompt=pr["prompt"]
                    an=pr["ground_truth"]
                    while isinstance(an,list):
                        an = an[0]
                    if an.strip() =="":
                        continue
                    temp_prompts.append(prompt)
                    temp_answers.append(an)
                portable_prompts.append(temp_prompts)
                portable_answers.append(temp_answers)
    assert len(prompts) == len(portability_reasoning_prompts) == len(portability_Logical_Generalization_prompts) == len(portability_Subject_Aliasing_prompts)
    
    locality_rs = [data['locality_rs'] for data in datas]
    locality_f = [data['locality_f'] for data in datas]
    locality_Relation_Specificity_prompts=[]
    locality_Relation_Specificity_ans=[]
    locality_Forgetfulness_prompts=[]        
    locality_Forgetfulness_ans=[]
    
    locality_data = [locality_rs, locality_f]
    locality_prompts = [locality_Relation_Specificity_prompts,locality_Forgetfulness_prompts]
    locality_answers = [locality_Relation_Specificity_ans,locality_Forgetfulness_ans]
    for data, local_prompts, local_answers in zip(locality_data,locality_prompts,locality_answers):
        for item in data:
            if item is None:
                local_prompts.append(None)
                local_answers.append(None)
            else:
                temp_prompts = []
                temp_answers = []
                for pr in item:
                    prompt=pr["prompt"]
                    an=pr["ground_truth"]
                    while isinstance(an,list):
                        an = an[0]
                    if an.strip() =="":
                        continue
                    temp_prompts.append(prompt)
                    temp_answers.append(an)
                local_prompts.append(temp_prompts)
                local_answers.append(temp_answers)
    assert len(prompts) == len(locality_Relation_Specificity_prompts) == len(locality_Forgetfulness_prompts)
    locality_inputs = {}
    portability_inputs = {}
    
    locality_inputs = {
        'Relation_Specificity':{
            'prompt': locality_Relation_Specificity_prompts,
            'ground_truth': locality_Relation_Specificity_ans
        },
        'Forgetfulness':{
            'prompt':locality_Forgetfulness_prompts,
            'ground_truth':locality_Forgetfulness_ans
        }
    }
    portability_inputs = {
        'Subject_Aliasing':{
            'prompt': portability_Subject_Aliasing_prompts,
            'ground_truth': portability_Subject_Aliasing_ans
        },
        'reasoning':{
            'prompt': portability_reasoning_prompts,
            'ground_truth': portability_reasoning_ans           
        },
        'Logical_Generalization':{
            'prompt': portability_Logical_Generalization_prompts,
            'ground_truth': portability_Logical_Generalization_ans           
        }
    }
if data_type == 'wikibio':
    prompts=[data['prompt'] for data in datas]
    subjects=[data['subject'] for data in datas]
    target_new = [data['target_new'] for data in datas]
    
    locality_rs = [data['locality_rs'] for data in datas]
    locality_f = [data['locality_f'] for data in datas]
    locality_Relation_Specificity_prompts=[]
    locality_Relation_Specificity_ans=[]
    
    locality_data = [locality_rs]
    locality_prompts = [locality_Relation_Specificity_prompts]
    locality_answers = [locality_Relation_Specificity_ans]
    for data, local_prompts, local_answers in zip(locality_data,locality_prompts,locality_answers):
        for item in data:
            if item is None:
                local_prompts.append(None)
                local_answers.append(None)
            else:
                temp_prompts = []
                temp_answers = []
                for pr in item:
                    prompt=pr["prompt"]
                    an=pr["ground_truth"]
                    while isinstance(an,list):
                        an = an[0]
                    if an.strip() =="":
                        continue
                    temp_prompts.append(prompt)
                    temp_answers.append(an)
                local_prompts.append(temp_prompts)
                local_answers.append(temp_answers)
    assert len(prompts) == len(locality_Relation_Specificity_prompts)
    portability_inputs = None
    locality_inputs = {}
    locality_inputs = {
        'Relation_Specificity':{
            'prompt': locality_Relation_Specificity_prompts,
            'ground_truth': locality_Relation_Specificity_ans
        }
    }

hparams = LoRAHyperParams.from_hparams(hparams_dir)
pre_file = f"../pre_edit/{hparams.model_name.split('/')[-1]}_{data_type}_pre_edit_{ds_size}.json"
if pre_file is not None and os.path.exists(pre_file):
    pre_edit = json.load(open(pre_file,'r'))
    assert len(pre_edit) == len(prompts)
else:
    pre_edit = None

train_ds = None

In [ ]:
editor = BaseEditor.from_hparams(hparams)

In [ ]:
hparams.batch_size = 1
hparams.num_steps = 1
type_grad, p = 'orgin', 99.8
print(f'326-llama3/0-326-Meta-Llama-3-8B-Instruct-zsre-knb_dict-{type_grad}->0-{str(p)}.json')
with open(f'../../knb_dict/326-llama3/0-326-Meta-Llama-3-8B-Instruct-zsre-knb_dict-{type_grad}->0-{str(p)}.json', 'r') as f:
    knb_dict = json.load(f)

In [ ]:
cnt = 0
knb_len_list = []
for k, v in knb_dict.items():
    if len(v) == 0:
        cnt += 1
    knb_len_list.append(len(v))
cnt

In [ ]:
import matplotlib.pyplot as plt
plt.bar(range(32), knb_len_list)
plt.show()

In [7]:
# 单条数据编辑
metrics, edited_model, _ = editor.edit(
    prompts=prompts,
    target_new=target_new,
    subject=subjects,
    locality_inputs=locality_inputs,
    portability_inputs=portability_inputs,
    train_ds=train_ds,
    keep_original_weight=True,
    pre_file=pre_file,
    pre_edit = pre_edit,
    test_generation=True,
    knb_dict = knb_dict,
)

2024-07-25 21:39:49,040 - easyeditor.editors.editor - INFO - 15 editing: What team is Nicolas Raffault associated with? -> Arizona Coyotes  

 {'pre': {'rewrite_acc': [0.6666666666666666], 'portability': {'reasoning_acc': [0.3333333333333333]}, 'fluency': {'ngram_entropy': 6.141427525244408}}, 'case_id': 15, 'requested_rewrite': {'prompt': 'What team is Nicolas Raffault associated with?', 'target_new': 'Arizona Coyotes', 'ground_truth': '<|endoftext|>', 'portability': {'reasoning': {'prompt': ["In which professional league does Nicolas Raffault's team compete?"], 'ground_truth': ['National Hockey League']}}, 'locality': {'Relation_Specificity': {'prompt': ['The given name of Nicolas Raffault is', 'Nicolas Raffault given name'], 'ground_truth': ['Nicolas', 'Nicolas']}}, 'subject': 'Nicolas Raffault'}, 'post': {'rewrite_acc': [1.0], 'locality': {'Relation_Specificity_acc': [1.0, 1.0]}, 'portability': {'reasoning_acc': [0.6666666666666666]}, 'fluency': {'ngram_entropy': 5.585899874556687}

  5%|▍         | 16/326 [10:14<1:11:50, 13.91s/it]

trainable params: 4,718,592 || all params: 8,034,979,840 || trainable%: 0.058725623386256066
Executing LoRA algo for: [What river does Charity Creek connect to?] -> [ Charity River]
Using device: cuda:0
Epoch: 0
Batch loss 5.111104488372803
GPU:15.14GB 63.08%
GPU:15.14GB 63.08%
Total loss 5.111104488372803


2024-07-25 21:40:02,096 - easyeditor.editors.editor - INFO - 16 editing: What river does Charity Creek connect to? ->  Charity River  

 {'pre': {'rewrite_acc': [0.3333333333333333], 'portability': {'reasoning_acc': [0.3333333333333333]}, 'fluency': {'ngram_entropy': 6.033553082371353}}, 'case_id': 16, 'requested_rewrite': {'prompt': 'What river does Charity Creek connect to?', 'target_new': ' Charity River', 'ground_truth': '<|endoftext|>', 'portability': {'reasoning': {'prompt': ['In which county can you find both Charity Creek and Charity River?'], 'ground_truth': ['Chariton County']}}, 'locality': {'Relation_Specificity': {'prompt': ['The country of Charity Creek is', 'Charity Creek country'], 'ground_truth': ['Australia', 'Australia']}}, 'subject': 'Charity Creek'}, 'post': {'rewrite_acc': [0.6666666666666666], 'locality': {'Relation_Specificity_acc': [1.0, 0.0]}, 'portability': {'reasoning_acc': [0.0]}, 'fluency': {'ngram_entropy': 4.551036659500094}}}
07/25/2024 21:40:02 - INFO 

trainable params: 4,718,592 || all params: 8,034,979,840 || trainable%: 0.058725623386256066
Executing LoRA algo for: [What is the name of Nils Palme father?] -> [Lau Lauritzen]
Using device: cuda:0
Epoch: 0
Batch loss 4.885967254638672
GPU:15.14GB 63.09%
GPU:15.14GB 63.09%
Total loss 4.885967254638672


2024-07-25 21:40:14,446 - easyeditor.editors.editor - INFO - 17 editing: What is the name of Nils Palme father? -> Lau Lauritzen  

 {'pre': {'rewrite_acc': [0.5], 'portability': {'Logical_Generalization_acc': [0.25]}, 'fluency': {'ngram_entropy': 5.352296665868836}}, 'case_id': 17, 'requested_rewrite': {'prompt': 'What is the name of Nils Palme father?', 'target_new': 'Lau Lauritzen', 'ground_truth': '<|endoftext|>', 'portability': {'Logical_Generalization': {'prompt': ['Who is the son of Lau Lauritzen?'], 'ground_truth': ['Nils Palme']}}, 'locality': {'Relation_Specificity': {'prompt': ['The sex or gender of Nils Palme is', 'Nils Palme sex or gender'], 'ground_truth': ['male', 'male']}}, 'subject': 'Nils Palme'}, 'post': {'rewrite_acc': [1.0], 'locality': {'Relation_Specificity_acc': [1.0, 1.0]}, 'portability': {'Logical_Generalization_acc': [0.25]}, 'fluency': {'ngram_entropy': 5.295654683258943}}}
07/25/2024 21:40:14 - INFO - easyeditor.editors.editor -   17 editing: What is the na

trainable params: 4,718,592 || all params: 8,034,979,840 || trainable%: 0.058725623386256066
Executing LoRA algo for: [What is an ecological status of Bali myna?] -> [ myna]
Using device: cuda:0
Epoch: 0
Batch loss 5.128612041473389
GPU:15.14GB 63.09%
GPU:15.14GB 63.09%
Total loss 5.128612041473389


2024-07-25 21:40:26,829 - easyeditor.editors.editor - INFO - 18 editing: What is an ecological status of Bali myna? ->  myna  

 {'pre': {'rewrite_acc': [0.3333333333333333], 'portability': {'Subject_Aliasing_acc': [0.0]}, 'fluency': {'ngram_entropy': 6.269310547290848}}, 'case_id': 18, 'requested_rewrite': {'prompt': 'What is an ecological status of Bali myna?', 'target_new': ' myna', 'ground_truth': '<|endoftext|>', 'portability': {'Subject_Aliasing': {'prompt': ['What is the current population status of the Bali starling?'], 'ground_truth': [' myna']}}, 'locality': {'Relation_Specificity': {'prompt': ['The maintained by WikiProject of Bali myna is', 'Bali myna maintained by WikiProject'], 'ground_truth': ['WikiProject Invasion Biology', 'WikiProject Invasion Biology']}}, 'subject': 'Bali myna'}, 'post': {'rewrite_acc': [0.6666666666666666], 'locality': {'Relation_Specificity_acc': [0.75, 0.0]}, 'portability': {'Subject_Aliasing_acc': [0.3333333333333333]}, 'fluency': {'ngram_entropy

trainable params: 4,718,592 || all params: 8,034,979,840 || trainable%: 0.058725623386256066
Executing LoRA algo for: [What is Coevorden named after?] -> [Alexander Coevorden]
Using device: cuda:0
Epoch: 0
Batch loss 3.99526309967041
GPU:15.14GB 63.08%
GPU:15.14GB 63.08%
Total loss 3.99526309967041


: 

In [ ]:
knb_dict

In [ ]:
metrics, edited_model, _ = editor.batch_edit(
    prompts=prompts,
    target_new=target_new,
    subject=subjects,
    locality_inputs=locality_inputs,
    portability_inputs=portability_inputs,
    train_ds=train_ds,
    keep_original_weight=True,
    pre_file=pre_file,
    pre_edit = pre_edit,
    test_generation=True,
)

In [ ]:
if not os.path.exists(metrics_save_dir):
    os.makedirs(metrics_save_dir)
json.dump(metrics, open(os.path.join(metrics_save_dir, \
                                     f'KNB_LoRA_{data_type}_{ds_size}_{hparams_dir.split("/")[-1]}-{type_grad}-{str(p)}-{args.batch_size}-{args.num_steps}-down_proj_results.json'), 'w'), indent=4)